In [21]:
import MySQLdb
import os
import csv
import sys
import pandas as pd
import tqdm
import sqlalchemy
import re
project_home = os.environ['PACKAGE_HOME']
from Development.helpers import general_helpers
from datetime import datetime
import configparser
config = configparser.ConfigParser()
config.read(project_home + '/Development/config.ini')

disambig_folder = '{}/{}'.format(config['FOLDERS']['WORKING_FOLDER'],'disambig_output')
new_db = config['DATABASE']['NEW_DB']
old_db = config['DATABASE']['OLD_DB']

engine = general_helpers.connect_to_db(config['DATABASE']['HOST'], config['DATABASE']['USERNAME'], config['DATABASE']['PASSWORD'], new_db)
db_con = engine.connect() 


In [22]:
db = 'patent_20191231'
pid_tbl = 'persistent_inventor_disambig'

In [23]:
# get columns 
result = db_con.execute('show columns from {0}.{1}'.format(db, pid_tbl))
pid_cols = [x[0] for x in result]
print(pid_cols)

['current_rawinventor_id', 'old_rawinventor_id', 'disamb_inventor_id_20170808', 'disamb_inventor_id_20171003', 'disamb_inventor_id_20171226', 'disamb_inventor_id_20180528', 'disamb_inventor_id_20181127', 'disamb_inventor_id_20190312', 'disamb_inventor_id_20190820', 'disamb_inventor_id_20191008', 'disamb_inventor_id_20191231']


In [24]:
disambig_cols = [x for x in pid_cols if x.startswith('disamb')]
disambig_cols

['disamb_inventor_id_20170808',
 'disamb_inventor_id_20171003',
 'disamb_inventor_id_20171226',
 'disamb_inventor_id_20180528',
 'disamb_inventor_id_20181127',
 'disamb_inventor_id_20190312',
 'disamb_inventor_id_20190820',
 'disamb_inventor_id_20191008',
 'disamb_inventor_id_20191231']

### QA Metric 1:  Change in inventor ids between data updates

In [25]:
# get all pairs of counts to generate
pairs = []
for x in range(0,len(disambig_cols) - 1):
    pairs.append((disambig_cols[x], disambig_cols[x + 1]))

pairs

[('disamb_inventor_id_20170808', 'disamb_inventor_id_20171003'),
 ('disamb_inventor_id_20171003', 'disamb_inventor_id_20171226'),
 ('disamb_inventor_id_20171226', 'disamb_inventor_id_20180528'),
 ('disamb_inventor_id_20180528', 'disamb_inventor_id_20181127'),
 ('disamb_inventor_id_20181127', 'disamb_inventor_id_20190312'),
 ('disamb_inventor_id_20190312', 'disamb_inventor_id_20190820'),
 ('disamb_inventor_id_20190820', 'disamb_inventor_id_20191008'),
 ('disamb_inventor_id_20191008', 'disamb_inventor_id_20191231')]

In [26]:
changed_inv_counts = []
equal_inv_counts = []
for p in pairs:
    c1 = p[0]
    c2 = p[1]
    sql_stmt = "select count(*) from {0}.{1} where {2} <> {3} and {2} <> '';".format(db,pid_tbl, c1, c2)
    print(sql_stmt)
    result = db_con.execute(sql_stmt)
    res_lst = [x[0] for x in result]
    changed_inv_counts.append(res_lst[0])
    
    sql_stmt = 'select count(*) from {0}.{1} where {2} = {3};'.format(db,pid_tbl, c1, c2)
    print(sql_stmt)
    result = db_con.execute(sql_stmt)
    res_lst = [x[0] for x in result]
    equal_inv_counts.append(res_lst[0])
    
    print('----------------------------------------------------------------\n')

select count(*) from patent_20191231.persistent_inventor_disambig where disamb_inventor_id_20170808 <> disamb_inventor_id_20171003 and disamb_inventor_id_20170808 <> '';
select count(*) from patent_20191231.persistent_inventor_disambig where disamb_inventor_id_20170808 = disamb_inventor_id_20171003;
----------------------------------------------------------------

select count(*) from patent_20191231.persistent_inventor_disambig where disamb_inventor_id_20171003 <> disamb_inventor_id_20171226 and disamb_inventor_id_20171003 <> '';
select count(*) from patent_20191231.persistent_inventor_disambig where disamb_inventor_id_20171003 = disamb_inventor_id_20171226;
----------------------------------------------------------------

select count(*) from patent_20191231.persistent_inventor_disambig where disamb_inventor_id_20171226 <> disamb_inventor_id_20180528 and disamb_inventor_id_20171226 <> '';
select count(*) from patent_20191231.persistent_inventor_disambig where disamb_inventor_id_20171

In [27]:
ratio_ids_changed_equal = []
for x in range(0, len(changed_inv_counts)):
    ratio_ce = round(changed_inv_counts[x]/equal_inv_counts[x], 3)
    ratio_ids_changed_equal.append(ratio_ce)

In [12]:
c1 = [p[0].strip('disamb_inventor_id_') for p in pairs]
c2 = [p[1].strip('disamb_inventor_id_') for p in pairs]
df_dict = {'prev_db': c1, 'new_db':c2, 'n_changed_inv_ids':changed_inv_counts, 'n_same_inv_ids':equal_inv_counts,
          'ratio_ids_ce': ratio_ids_changed_equal}
df = pd.DataFrame(df_dict)
df



,prev_db,new_db,n_changed_inv_ids,n_same_inv_ids,ratio_ids_ce
0,20170808,20171003,926330,16347040,0.057
1,20171003,20171226,1819831,15389206,0.118
2,20171226,20180528,1768404,15242611,0.116
3,20180528,20181127,4077845,12864964,0.317
4,20181127,20190312,4609633,12544169,0.367
5,20190312,20190820,689637,16244416,0.042
6,20190820,20191008,586740,16683681,0.035
7,20191008,20191231,605575,16560029,0.037


In [15]:
df.to_csv('qa1_inv_id_change.csv', index=False)

### QA Metric 2: Disappearance of Inventors

In [30]:
# investigate existing _____ null _____ cases
middle_null_counts = []
col1 = []
col2 = []
pid_tbl = 'persistent_inventor_disambig_type_2'
for p in pairs[:-1]:
    prevdbupdate = p[0]
    newdbupdate = p[1]
    
    nested_query_stmt = "select concat(patent_id, '|', sequence) as identifier from {0}.{1} where {2} is not null);".format(db, pid_tbl, prevdbupdate)
    
    outer_query_stmt = "select count(*) as identifier from {0}.{1} where {2} is null and concat(patent_id, '|', sequence) in (".format(db, pid_tbl,newdbupdate)

    sql_stmt = outer_query_stmt + nested_query_stmt 
    print(sql_stmt)

    result = db_con.execute(sql_stmt)
    res_lst = [x[0] for x in result]
    middle_null_counts.append(res_lst[0])
    col1.append(prevdbupdate)
    col2.append(newdbupdate)

    print('----------------------------------------------------------------\n')


select count(*) as identifier from patent_20191231.persistent_inventor_disambig_type_2 where disamb_inventor_id_20171003 is null and concat(patent_id, '|', sequence) in (select concat(patent_id, '|', sequence) as identifier from patent_20191231.persistent_inventor_disambig_type_2 where disamb_inventor_id_20170808 is not null);
----------------------------------------------------------------

select count(*) as identifier from patent_20191231.persistent_inventor_disambig_type_2 where disamb_inventor_id_20171226 is null and concat(patent_id, '|', sequence) in (select concat(patent_id, '|', sequence) as identifier from patent_20191231.persistent_inventor_disambig_type_2 where disamb_inventor_id_20171003 is not null);
----------------------------------------------------------------

select count(*) as identifier from patent_20191231.persistent_inventor_disambig_type_2 where disamb_inventor_id_20180528 is null and concat(patent_id, '|', sequence) in (select concat(patent_id, '|', sequence) 

In [33]:

c1 = [x.strip('disamb_inventor_id_') for x in col1]
c2 = [x.strip('disamb_inventor_id_') for x in col2]
df_dict = {'prev_db': c1, 'new_db':c2, 'n_middlenulls':middle_null_counts}
df = pd.DataFrame(df_dict)
df


,prev_db,new_db,n_middlenulls
0,20170808,20171003,0
1,20171003,20171226,0
2,20171226,20180528,0
3,20180528,20181127,0
4,20181127,20190312,0
5,20190312,20190820,0
6,20190820,20191008,0


In [34]:
df.to_csv('qa2_inv_disappearance.csv', index=False)

### QA Metric 3: Size of the Top n Disambiguated Inventor Clusters

In [ ]:
# 118 seconds
full_df = pd.DataFrame()

for col in disambig_cols:
    stmt = 'select {0}, count(*) as n_cluster from {1}.{2} group by {0} order by n_cluster desc limit 21;'.format(col, db, pid_tbl)
    
    print(stmt)
    
    result_qry = db_con.execute(stmt)
    
    results = [x for x in result_qry]


    dbupdate_colname = col.strip('disamb_inventor_id_')
    ncluster_dbupdate = 'n_cluster' + '_' + dbupdate_colname

    db_inv_id = [r[0] for r in results]
    n_inv_cluster = [r[1] for r in results]

    # create df
    df_dict = {dbupdate_colname: db_inv_id, ncluster_dbupdate: n_inv_cluster}
    new_df = pd.DataFrame(df_dict)
    new_df

    # see full clusters across updates
    full_df = pd.concat([full_df, new_df], axis=1)
    print(full_df.head())




select disamb_inventor_id_20170808, count(*) as n_cluster from patent_20191231.persistent_inventor_disambig_type_2 group by disamb_inventor_id_20170808 order by n_cluster desc limit 21;
    20170808  n_cluster_20170808
0                        2206122
1  3967981-1                5046
2  5329616-1                4772
3  4632910-2                3847
4  4039851-2                3397
select disamb_inventor_id_20171003, count(*) as n_cluster from patent_20191231.persistent_inventor_disambig_type_2 group by disamb_inventor_id_20171003 order by n_cluster desc limit 21;


In [ ]:
full_df

In [ ]:
full_df.to_csv('top10inventorclusters_by_db.csv', index=False)

In [ ]:
os.getcwd()